<a href="https://colab.research.google.com/github/DevzsJhonny/Colab_notebooks/blob/main/Big_data_aula_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando framework **Pyspark**

In [27]:
!pip install pyspark

In [28]:
!pip install findspark

In [29]:
import findspark #importação
findspark.init() #inicialização
from pyspark.sql import SparkSession #importação de sessão de uso do spark em nosso notebook
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [30]:
df = spark.sql("""select 'spark' as hello""")
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [31]:
# Import spark libraries
from pyspark.sql import Row, DataFrame
from pyspark.sql.types import StringType, StructType, StructField, IntegerType
from pyspark.sql.functions import col, expr, lit, substring, concat, concat_ws, when, coalesce
from pyspark.sql import functions as F # for more sql functions
from functools import reduce

# Manipulação de dados com Spark

In [32]:

df = spark.read.csv('banklist.csv', sep = ',', inferSchema = True, header = True)

print('df.count  :', df.count())
print('df.col ct :', len(df.columns))
print('df.columns:', df.columns)

df.count  : 561
df.col ct : 6
df.columns: ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']


# Utilizando o SQL

In [33]:

df.createOrReplaceTempView("banklist")

df_check = spark.sql('''select `Bank Name`, City, `Closing Date` from banklist''')
#df_check.show(4, truncate=False)
df_check.show()

+--------------------+------------------+------------+
|           Bank Name|              City|Closing Date|
+--------------------+------------------+------------+
|The First State Bank|     Barboursville|    3-Apr-20|
|  Ericson State Bank|           Ericson|   14-Feb-20|
|City National Ban...|            Newark|    1-Nov-19|
|       Resolute Bank|            Maumee|   25-Oct-19|
|Louisa Community ...|            Louisa|   25-Oct-19|
|The Enloe State Bank|            Cooper|   31-May-19|
|Washington Federa...|           Chicago|   15-Dec-17|
|The Farmers and M...|           Argonia|   13-Oct-17|
| Fayette County Bank|        Saint Elmo|   26-May-17|
|Guaranty Bank, (d...|         Milwaukee|    5-May-17|
|      First NBC Bank|       New Orleans|   28-Apr-17|
|       Proficio Bank|Cottonwood Heights|    3-Mar-17|
|Seaway Bank and T...|           Chicago|   27-Jan-17|
|Harvest Community...|        Pennsville|   13-Jan-17|
|         Allied Bank|          Mulberry|   23-Sep-16|
|The Woodb

# Operações Básicas com DataFrame

In [34]:
df.describe().show()

+-------+--------------------+-------+----+-----------------+---------------------+------------+
|summary|           Bank Name|   City|  ST|             CERT|Acquiring Institution|Closing Date|
+-------+--------------------+-------+----+-----------------+---------------------+------------+
|  count|                 561|    561| 561|              561|                  561|         561|
|   mean|                NULL|   NULL|NULL|31685.68449197861|                 NULL|        NULL|
| stddev|                NULL|   NULL|NULL|16446.65659309965|                 NULL|        NULL|
|    min|1st American Stat...|Acworth|  AL|               91|      1st United Bank|    1-Aug-08|
|    max|               ebank|Wyoming|  WY|            58701|  Your Community Bank|    9-Sep-11|
+-------+--------------------+-------+----+-----------------+---------------------+------------+



In [35]:
df.describe('City','ST').show()

+-------+-------+----+
|summary|   City|  ST|
+-------+-------+----+
|  count|    561| 561|
|   mean|   NULL|NULL|
| stddev|   NULL|NULL|
|    min|Acworth|  AL|
|    max|Wyoming|  WY|
+-------+-------+----+



# Count, columns e Schema

In [36]:

print('Total de linhas:', df.count())
print('Total de colunas:', len(df.columns))
print('Tipo de dados:', df.dtypes)
print('Schema:', df.schema)

Total de linhas: 561
Total de colunas: 6
Tipo de dados: [('Bank Name', 'string'), ('City', 'string'), ('ST', 'string'), ('CERT', 'int'), ('Acquiring Institution', 'string'), ('Closing Date', 'string')]
Schema: StructType([StructField('Bank Name', StringType(), True), StructField('City', StringType(), True), StructField('ST', StringType(), True), StructField('CERT', IntegerType(), True), StructField('Acquiring Institution', StringType(), True), StructField('Closing Date', StringType(), True)])


# Removendo duplicidades

In [37]:

df = df.dropDuplicates()
print('df.count		:', df.count())
print('df.columns	:', df.columns)


df.count		: 561
df.columns	: ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']


# Select em colunas específicas

In [38]:
df2 = df.select(*['Bank Name', 'City'])
df2.show()


+--------------------+----------------+
|           Bank Name|            City|
+--------------------+----------------+
| First Bank of Idaho|         Ketchum|
|Amcore Bank, Nati...|        Rockford|
|        Venture Bank|           Lacey|
|First State Bank ...|           Altus|
|Valley Capital Ba...|            Mesa|
|Michigan Heritage...|Farmington Hills|
|Columbia Savings ...|      Cincinnati|
|       Fidelity Bank|        Dearborn|
|The Park Avenue Bank|        Valdosta|
|Western Commercia...|  Woodland Hills|
|        Syringa Bank|           Boise|
|Republic Federal ...|           Miami|
|Westside Communit...|University Place|
|   First United Bank|           Crete|
|HarVest Bank of M...|    Gaithersburg|
|            BankEast|       Knoxville|
|    Polk County Bank|        Johnston|
|Colorado Capital ...|     Castle Rock|
|         Access Bank|        Champlin|
|Pacific National ...|   San Francisco|
+--------------------+----------------+
only showing top 20 rows



# Select em colunas múltiplas

In [39]:
col_l = list(set(df.columns)  - {'CERT','ST'})
df2 = df.select(*col_l)
df2.show()


+--------------------+----------------+---------------------+------------+
|           Bank Name|            City|Acquiring Institution|Closing Date|
+--------------------+----------------+---------------------+------------+
| First Bank of Idaho|         Ketchum|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|        Rockford|          Harris N.A.|   23-Apr-10|
|        Venture Bank|           Lacey| First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|           Altus|         Herring Bank|   31-Jul-09|
|Valley Capital Ba...|            Mesa| Enterprise Bank &...|   11-Dec-09|
|Michigan Heritage...|Farmington Hills|       Level One Bank|   24-Apr-09|
|Columbia Savings ...|      Cincinnati| United Fidelity B...|   23-May-14|
|       Fidelity Bank|        Dearborn| The Huntington Na...|   30-Mar-12|
|The Park Avenue Bank|        Valdosta|   Bank of the Ozarks|   29-Apr-11|
|Western Commercia...|  Woodland Hills| First California ...|    5-Nov-10|
|        Syringa Bank|   

# Renomear colunas

In [40]:

df2 = df \
  .withColumnRenamed('Bank Name'            , 'bank_name') \
  .withColumnRenamed('Acquiring Institution', 'acq_institution') \
  .withColumnRenamed('Closing Date'         , 'closing_date') \
  .withColumnRenamed('ST'                   , 'state') \
  .withColumnRenamed('CERT'                 , 'cert') #\

df2.show()

+--------------------+----------------+-----+-----+--------------------+------------+
|           bank_name|            City|state| cert|     acq_institution|closing_date|
+--------------------+----------------+-----+-----+--------------------+------------+
| First Bank of Idaho|         Ketchum|   ID|34396|     U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|        Rockford|   IL| 3735|         Harris N.A.|   23-Apr-10|
|        Venture Bank|           Lacey|   WA|22868|First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|           Altus|   OK| 9873|        Herring Bank|   31-Jul-09|
|Valley Capital Ba...|            Mesa|   AZ|58399|Enterprise Bank &...|   11-Dec-09|
|Michigan Heritage...|Farmington Hills|   MI|34369|      Level One Bank|   24-Apr-09|
|Columbia Savings ...|      Cincinnati|   OH|32284|United Fidelity B...|   23-May-14|
|       Fidelity Bank|        Dearborn|   MI|33883|The Huntington Na...|   30-Mar-12|
|The Park Avenue Bank|        Valdosta|   GA|19797|  B

# Adicionando Colunas

In [41]:
df2 = df.withColumn('state', col('ST'))
df2.show()

+--------------------+----------------+---+-----+---------------------+------------+-----+
|           Bank Name|            City| ST| CERT|Acquiring Institution|Closing Date|state|
+--------------------+----------------+---+-----+---------------------+------------+-----+
| First Bank of Idaho|         Ketchum| ID|34396|      U.S. Bank, N.A.|   24-Apr-09|   ID|
|Amcore Bank, Nati...|        Rockford| IL| 3735|          Harris N.A.|   23-Apr-10|   IL|
|        Venture Bank|           Lacey| WA|22868| First-Citizens Ba...|   11-Sep-09|   WA|
|First State Bank ...|           Altus| OK| 9873|         Herring Bank|   31-Jul-09|   OK|
|Valley Capital Ba...|            Mesa| AZ|58399| Enterprise Bank &...|   11-Dec-09|   AZ|
|Michigan Heritage...|Farmington Hills| MI|34369|       Level One Bank|   24-Apr-09|   MI|
|Columbia Savings ...|      Cincinnati| OH|32284| United Fidelity B...|   23-May-14|   OH|
|       Fidelity Bank|        Dearborn| MI|33883| The Huntington Na...|   30-Mar-12|   MI|

#Adicionando coluna constante (valor fixo)

In [42]:
df2 = df.withColumn('country', lit('US'))
df2.show()

+--------------------+----------------+---+-----+---------------------+------------+-------+
|           Bank Name|            City| ST| CERT|Acquiring Institution|Closing Date|country|
+--------------------+----------------+---+-----+---------------------+------------+-------+
| First Bank of Idaho|         Ketchum| ID|34396|      U.S. Bank, N.A.|   24-Apr-09|     US|
|Amcore Bank, Nati...|        Rockford| IL| 3735|          Harris N.A.|   23-Apr-10|     US|
|        Venture Bank|           Lacey| WA|22868| First-Citizens Ba...|   11-Sep-09|     US|
|First State Bank ...|           Altus| OK| 9873|         Herring Bank|   31-Jul-09|     US|
|Valley Capital Ba...|            Mesa| AZ|58399| Enterprise Bank &...|   11-Dec-09|     US|
|Michigan Heritage...|Farmington Hills| MI|34369|       Level One Bank|   24-Apr-09|     US|
|Columbia Savings ...|      Cincinnati| OH|32284| United Fidelity B...|   23-May-14|     US|
|       Fidelity Bank|        Dearborn| MI|33883| The Huntington Na...

#Drop colunas

In [43]:
df2 = df.drop('CERT')
df2.show(2)

+--------------------+--------+---+---------------------+------------+
|           Bank Name|    City| ST|Acquiring Institution|Closing Date|
+--------------------+--------+---+---------------------+------------+
| First Bank of Idaho| Ketchum| ID|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|Rockford| IL|          Harris N.A.|   23-Apr-10|
+--------------------+--------+---+---------------------+------------+
only showing top 2 rows



# Dropando mais de uma coluna

In [44]:
df2 = df.drop(*['CERT','ST'])
df2.show(2)
# or df2 = reduce(DataFrame.drop, ['CERT','ST'], df)


+--------------------+--------+---------------------+------------+
|           Bank Name|    City|Acquiring Institution|Closing Date|
+--------------------+--------+---------------------+------------+
| First Bank of Idaho| Ketchum|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|Rockford|          Harris N.A.|   23-Apr-10|
+--------------------+--------+---------------------+------------+
only showing top 2 rows



#Filtrando data

In [45]:
# Equal to values
df2 = df.where(df['ST'] == 'NE')

# Between values
df3 = df.where(df['CERT'].between('1000','2000'))

# Is inside multiple values
df4 = df.where(df['ST'].isin('NE','IL'))

print('df.count  :', df.count())
print('df2.count :', df2.count())
print('df3.count :', df3.count())
print('df4.count :', df4.count())

df.count  : 561
df2.count : 4
df3.count : 9
df4.count : 73


# Operadores Lógicos

In [46]:
df2 = df.where((df['ST'] == 'NE') & (df['City'] == 'Ericson'))
df2.show(3)

+------------------+-------+---+-----+---------------------+------------+
|         Bank Name|   City| ST| CERT|Acquiring Institution|Closing Date|
+------------------+-------+---+-----+---------------------+------------+
|Ericson State Bank|Ericson| NE|18265| Farmers and Merch...|   14-Feb-20|
+------------------+-------+---+-----+---------------------+------------+



# replace

In [48]:
# Pre replace
print('Pre replace')
df.show(2)

# Post replace
print('Pos replace')

print('Replace 7 in the above dataframe with 17 at all instances')
df.na.replace(7,17).show()

Pre replace
+--------------------+--------+---+-----+---------------------+------------+
|           Bank Name|    City| ST| CERT|Acquiring Institution|Closing Date|
+--------------------+--------+---+-----+---------------------+------------+
| First Bank of Idaho| Ketchum| ID|34396|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|Rockford| IL| 3735|          Harris N.A.|   23-Apr-10|
+--------------------+--------+---+-----+---------------------+------------+
only showing top 2 rows

Pos replace
Replace 7 in the above dataframe with 17 at all instances
+--------------------+----------------+---+-----+---------------------+------------+
|           Bank Name|            City| ST| CERT|Acquiring Institution|Closing Date|
+--------------------+----------------+---+-----+---------------------+------------+
| First Bank of Idaho|         Ketchum| ID|34396|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|        Rockford| IL| 3735|          Harris N.A.|   23-Apr-10|
|     